In [48]:
import json
from shapely.geometry import shape, Point

#to plot polygons
import matplotlib.pyplot as plt
import geopandas as gpd

#to open the csv, excel files as dataframes 
import pandas as pd
import numpy as np

#to list all files in a dir
from os import listdir, remove
from os.path import isfile, join

def check_single(geojs, point):
    for feature in geojs['features']:
        polygon = shape(feature['geometry'])
        #print(polygon)
        #p = gpd.GeoSeries(polygon)
        #p.plot()
        #plt.show()
        if polygon.contains(point):
            return True
        
    return False

def check(geofilesdir, spatialfilesdir):
    
    if(isfile(spatialfilesdir+"counts grouped.csv")):
        remove(spatialfilesdir+"counts grouped.csv")
        print("old counts grouped.csv deleted.")
    else:
        print("old counts grouped.csv not in dir")
    if(isfile(spatialfilesdir+"counts.csv")):
        remove(spatialfilesdir+"counts.csv")
        print("old counts.csv deleted.")
    else:
        print("old counts.csv not in dir")
    
    geofiles = [f for f in listdir(geofilesdir) if isfile(join(geofilesdir, f))]
    spatialfiles = [f for f in listdir(spatialfilesdir) if isfile(join(spatialfilesdir, f))]
    print(geofiles)
    print(spatialfiles)
    
    #new df to count all occurences
    counts = dict()
    
    for spatialfile in spatialfiles:
        print("Processing now: " + spatialfile)
        varname = spatialfile[8:len(spatialfile) - 4]
        df = pd.read_csv(spatialfilesdir+spatialfile)
        df["school_zones"] = ""
        
        for geofile in geofiles:
            with open(geofilesdir+geofile, 'r') as f:
                geojs = json.load(f)
            schoolname = geofile[:len(geofile) - 8]
            
            for index, row in df.iterrows():
                longitude = float(row['LONGITUDE'])
                latitude = float(row['LATITUDE'])
                point = Point(longitude, latitude)
                zones = row["school_zones"]
                if check_single(geojs, point):
                    zones = zones + schoolname + ","
                    
                    if (varname, schoolname + " School") in counts:
                        counts[(varname,schoolname+ " School")]+=1
                    else:
                        counts[(varname,schoolname + " School")] = 1
                    
                df.at[index,'school_zones'] = zones
        
        for index, row in df.iterrows():
            zones = row["school_zones"]
            zones = zones[:-1]
            df.at[index,'school_zones'] = zones
        
        newspatialfile = spatialfile
        df.to_csv(spatialfilesdir+newspatialfile)
        
    counts_df = pd.DataFrame(counts.values(), index=pd.MultiIndex.from_tuples(counts.keys())).unstack(1)
    counts_df.reset_index(inplace=True, col_level = 1)
    counts_df = counts_df.rename(columns = {'index':'varname'})
    counts_df = counts_df.fillna(0)
    counts_df.to_csv(spatialfilesdir+"counts.csv", header=1, index = False)
    
    counts_df = pd.read_csv(spatialfilesdir+"counts.csv", header=[1])
    counts_df = counts_df.melt(id_vars=["varname"], var_name="name", value_name="count")
    display(counts_df)
    counts_df = counts_df.fillna(0)
    counts_df.to_csv(spatialfilesdir+"counts grouped.csv")

In [49]:
#single example: Forest hills clubhouse contained in the CC Spaulding school zone
point = Point(-78.91356462, 35.97993926)
with open('data/2021/map_data/C.C. Spaulding Elementary.geojson', 'r') as f:
    geojs = json.load(f)
    
df = pd.read_csv("data/2021/spatial_data/renamed_Recreation Centers.csv")
point2 = Point(df.iloc[[11]]["LONGITUDE"], df.iloc[[11]]["LATITUDE"])

#should print True
print(check_single(geojs, point))
print(check_single(geojs, point2))

True
True


In [50]:
#updates the .csv files in spatial_data for all the specified years by adding a column 'school_zones' which contains all the
#school zones that resource is in.

years = ['2021']

for year in years:
    geofilesdir = 'data/'+year+'/map_data/'
    spatialfilesdir = 'data/'+year+'/spatial_data/'
    
    check(geofilesdir, spatialfilesdir)

old counts grouped.csv deleted.
old counts.csv deleted.
['All.geojson', 'Brogden Middle.geojson', 'C.C. Spaulding Elementary.geojson', 'Club Boulevard Elementary.geojson', 'E.K. Powe Elementary.geojson', 'Eastway Elementary.geojson', 'Fayetteville Street Elementary.geojson', 'Forest View Elementary.geojson', 'Hillandale Elementary.geojson', 'Hillside High.geojson', 'Jordan High.geojson', 'Lakewood Elementary.geojson', 'Lakewood Middle.geojson', 'Lowes Grove Middle.geojson', 'Parkwood Elementary.geojson', 'Riverside High.geojson', 'Southwest Elementary.geojson']
['renamed_After-School Care Programs.csv', 'renamed_Bus Stops.csv', 'renamed_Childcare Centers_2022.csv', 'renamed_Community & Cultural Centers.csv', 'renamed_Community Gardens.csv', "renamed_Farmer's Markets.csv", 'renamed_Food Pantries.csv', 'renamed_Grocery Stores.csv', 'renamed_Hospitals and Clinics.csv', 'renamed_Libraries.csv', 'renamed_Parks.csv', 'renamed_Recreation Centers.csv', 'renamed_Religious Centers.csv']
Processi

,varname,name,count
0,After-School Care Programs,All School,24.0
1,Bus Stops,All School,1037.0
2,Childcare Centers_2022,All School,258.0
3,Community & Cultural Centers,All School,9.0
4,Community Gardens,All School,6.0
...,...,...,...
216,Hospitals and Clinics,Southwest Elementary School,0.0
217,Libraries,Southwest Elementary School,0.0
218,Parks,Southwest Elementary School,3.0
219,Recreation Centers,Southwest Elementary School,0.0
